In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parameters
train_dir = './data/Train'  # Path to training dataset
test_dir = './data/Test'    # Path to testing dataset
batch_size = 32
epochs = 10
learning_rate = 0.001

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [5]:
# Load training data
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Load testing data
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [6]:
# Load pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Modify the final layer for binary classification
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
model = model.to(device)


c:\Users\KIIT0001\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\KIIT0001\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [8]:
# Training function
def train(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Validation function
def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


In [10]:
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, test_loader, criterion)
    
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Test Loss: {val_loss:.4f}, Test Accuracy: {val_acc:.4f}")


Epoch 1/10
Train Loss: 0.2963, Train Accuracy: 0.9563
Test Loss: 1.1812, Test Accuracy: 0.8562
Epoch 2/10
Train Loss: 0.1625, Train Accuracy: 0.9688
Test Loss: 0.1477, Test Accuracy: 0.9500
Epoch 3/10
Train Loss: 0.0752, Train Accuracy: 0.9672
Test Loss: 0.2433, Test Accuracy: 0.9000
Epoch 4/10
Train Loss: 0.2655, Train Accuracy: 0.9532
Test Loss: 0.3885, Test Accuracy: 0.7937
Epoch 5/10
Train Loss: 0.1096, Train Accuracy: 0.9672
Test Loss: 0.1632, Test Accuracy: 0.9437
Epoch 6/10
Train Loss: 0.1715, Train Accuracy: 0.9813
Test Loss: 0.0914, Test Accuracy: 0.9750
Epoch 7/10
Train Loss: 0.1449, Train Accuracy: 0.9860
Test Loss: 0.3998, Test Accuracy: 0.9625
Epoch 8/10
Train Loss: 0.1313, Train Accuracy: 0.9813
Test Loss: 0.3252, Test Accuracy: 0.8625
Epoch 9/10
Train Loss: 0.0792, Train Accuracy: 0.9797
Test Loss: 0.3909, Test Accuracy: 0.8500
Epoch 10/10
Train Loss: 0.1908, Train Accuracy: 0.9891
Test Loss: 0.0786, Test Accuracy: 0.9750
